In [1]:
import langchain
langchain.__version__ 

'0.1.13'

In [2]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

search_tool = Tool(
    name = "DuckDuckGo Search",
    func= DuckDuckGoSearchRun().run,
    description= "Use this tool to perform web searches using DuckDuckGo"
)

llm = OpenAI(temperature = 0.5)

tool = [search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the latest top raning movies in 2025?")

print(response)

c:\Anaconda3\envs\agentdemo\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Anaconda3\envs\agentdemo\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Anaconda3\envs\agentdemo\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  wa



> Entering new AgentExecutor chain...
 I should use DuckDuckGo Search to find the answer.
Action: DuckDuckGo Search
Action Input: "latest top ranking movies 2025"

c:\Anaconda3\envs\agentdemo\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:48: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Observation: Better Man (2024). Release Date: January 10, 2025 (after limited release on 12/25/24) Director: Michael Gracey Starring: Jonno Davies, Steve Pemberton, Damon Herriman, Kate Mulvany, Alison Steadman Micheal Gracey, director of The Greatest Showman, takes the reins in a biopic about UK musician Robbie Williams.The film is an original musical based on the life and career experiences of Williams ... Gooodbye, 2024, you year of belated sequels, failed superhero-movie spin-offs, sci-fi extravaganzas and a hit witch musical, you! Hello, 2025, the upcoming annum that promises more, er, belated ... "Captain America: Brave New World" is 2025's top pick in Fandango's annual Most Anticipated Movies survey of over 9,000 ticket-buyers. ... The 2025 Rotten Tomatoes Awards Honoring the best movies & TV; Go to next offer. 2025's Most Anticipated Movies. Fandango Staff December 11th 2024, 9:00 am. Share. Share on Facebook; Share on Twitter ... This film could end up being a stinker, leavin

# Wikipedia Search

In [5]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_search_tool = Tool(
    name = "Wikipedia Search",
    func= WikipediaAPIWrapper().run,
    description= "Use this tool to search Wikipedia for information on any topic."
)

llm = OpenAI(temperature = 0.5)

tool = [wiki_search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("Who is Elone musk? Tell me the latest news of him")

print(response)



> Entering new AgentExecutor chain...
 I should use Wikipedia Search to find information about Elon Musk
Action: Wikipedia Search
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and conservative political figure  known for his key roles in the automotive company Tesla, Inc. and the space company SpaceX. He is also known for his ownership of the technology company X Corp. and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. Musk is the wealthiest individual in the world; as of January 2025, Forbes estimates his net worth to be US$426 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria. At the age of 18 he immigrated to Canada, acquiring its citizenship through his Canadian-born mother, Maye. Two years later, he matriculated at Queen's University in Canada. Musk later transferred to the University of Pennsylva

# SQLite database

In [6]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [7]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 

In [8]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example_new.db")

# Step 3: Initialize the LLM
llm = OpenAI(temperature=0.5)

# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
 We need to query the database for the number of units of Product A that are currently in stock.
Action: sql_db_query
Action Input: SELECT stock FROM products WHERE name = 'Product A'
Observation: Error: (sqlite3.OperationalError) no such table: products
[SQL: SELECT stock FROM products WHERE name = 'Product A']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: The error message indicates that the table 'products' does not exist. We need to use sql_db_list_tables to get a list of all the tables in the database.
Action: sql_db_list_tables
Action Input: 
Observation: inventory
Thought: The only table in the database is 'inventory'. We need to use sql_db_schema to get the schema and sample rows for this table.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	pro

# ArxivQueryRun

In [9]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [10]:
arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

llm = OpenAI(temperature=0.5)

tools = [arxiv_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query = "Search for research papers on Object Detection in 2024."
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
 I should try using Arxiv Search to find relevant papers.
Action: Arxiv Search
Action Input: Object Detection 2024
Observation: Published: 2024-07-08
Title: Towards Reflected Object Detection: A Benchmark
Authors: Zhongtian Wang, You Wu, Hui Zhou, Shuiwang Li
Summary: Object detection has greatly improved over the past decade thanks to advances
in deep learning and large-scale datasets. However, detecting objects reflected
in surfaces remains an underexplored area. Reflective surfaces are ubiquitous
in daily life, appearing in homes, offices, public spaces, and natural
environments. Accurate detection and interpretation of reflected objects are
essential for various applications. This paper addresses this gap by
introducing a extensive benchmark specifically designed for Reflected Object
Detection. Our Reflected Object Detection Dataset (RODD) features a diverse
collection of images showcasing reflected objects in various contexts,
providing stan

# File Agents

In [11]:
from langchain_community.tools.file_management import ReadFileTool
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

In [12]:
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the content of a text file in the current working directory. Provide the file name as input."
)

llm = OpenAI(temperature=0.5)


tools = [read_file_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

file_name = "example.txt"

response = agent.run(f"Read the content of the file named {file_name}.")
print("Response from Agent:")
print(response)




> Entering new AgentExecutor chain...
 You should always think about what to do before taking any action.
Action: Read File
Action Input: example.txt
Observation: dswithbappy focuses on providing content on Data Science, Artificial Intelligence, Machine Learning, Deep Learning, Computer Vision, Natural language processing, Python programming, etc. in English. "Real creativity won't make things more complex. Instead, I will simplify them." I have been working in the Data Science field, spearheading in machine learning, deep learning, and computer vision. Typically this is my YouTube channel where I clarify different themes on machine learning, deep learning, and AI with numerous real-world issue scenarios. My primary point is to create everybody commonplace of ML and AI. Please subscribe and support the channel. As I come up with new innovations. These recordings are free and I guarantee to create more curiously substance as we go ahead.
😀Please donate if you want to support the chann